<a href="https://colab.research.google.com/github/OnyangoOmondie97/predict_health_costs_with_regression/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Check the column names
print("Columns before one-hot encoding:", dataset.columns)

# Converting categorical data to numerical values using one-hot encoding
columns_to_encode = ['sex', 'smoker', 'region']

for col in columns_to_encode:
    if col not in dataset.columns:
        print(f"Error: {col} column not found in the dataset.")
    else:
        dataset = pd.get_dummies(dataset, columns=[col], drop_first=True)

# Check the column names after one-hot encoding
print("Columns after one-hot encoding:", dataset.columns)

# Check if 'expenses' column is present
if 'expenses' not in dataset.columns:
    print("Error: 'expenses' column not found in the dataset after one-hot encoding.")
else:
    # Split the data into features (X) and labels (y)
    X = dataset.drop('expenses', axis=1)
    y = dataset['expenses']

    # Split the data into training and testing sets
    train_dataset, test_dataset, train_labels, test_labels = train_test_split(X, y, test_size=0.2, random_state=42)

    # Building the linear regression model
    model = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=[len(train_dataset.columns)]),
        layers.Dense(1)
    ])

    # Compiling the model
    model.compile(optimizer='adam', loss='mae')

    # Training the model
    history = model.fit(
        train_dataset, train_labels,
        epochs=100, validation_split=0.2, verbose=0,
        callbacks=[tfdocs.modeling.EpochDots()]
    )


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss = model.evaluate(test_dataset, test_labels, verbose=2)
mae = loss
mse = 0.0


print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
    print("You passed the challenge. Great job!")
else:
    print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)